# Assignment 2: Elasticsearch and Text Mining

Due: Monday, December 12, 2022, at 2pm via Moodle

**Team Members** `<Fill out>`

### Submission Guidelines

- Solutions need to be uploaded as a single Jupyter notebook. You will find this notebook contains some pre-filled cells that you should complete for the individual tasks.
- For answers requiring written solutions, use Markdown cells (in combination with Jupyter LaTeX support) **inside this notebook**. Do *not* hand in any separate files, simply re-upload the `.ipynb` file.
- Download the .zip file containing the dataset but do *not* upload it with your solution.
- Make sure that the names of all team members are present in the solution.

In [4]:
%%bash
. ~/.bashrc
python3 -m pip install -r requirements.txt
# Install necessary packages

In [13]:
%%bash
./ ~/.bashrc
python3 -m spacy download en_core_web_sm
# Install required language model for spaCy

bash: line 1: ./: Is a directory


  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.4.0
    Uninstalling en-core-web-sm-3.4.0:
      Successfully uninstalled en-core-web-sm-3.4.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



## Question 2-1: "The Simpsons" Dataset (1 + 4 + 3 + 2 + 6 + 5 = 21 points)

"The Simpsons" is the world’s longest-running animated sitcom. The show revolves around the lives of the Simpson family, which consists of Homer, Marge, Bart, Lisa, and the little Maggi.
For this exercise, you will work with the dataset that contains characters, locations, episode details, and script lines for approximately 600 Simpsons episodes, dating back to 1989.
We will explore the data and use visualizations to gain some insight into the show. The Zip file alongside this notebook should contain the following files:

` simpsons_characters.csv, simpsons_locations.csv, simpsons_script_lines.csv, simpsons_episodes.csv`

Take some time and familiarize yourself with the data. `simpsons_characters.csv` contains the character-names and their gender.
`simpsons_locations.csv` contains information about locations in the shows, and `simpsons_episodes.csv` has information about each episode including their title, rating, and views.
Finally, `simpsons_script_lines.csv` lists all the lines in the show, who said the line and where it was said. Be aware that this data set is not cleaned and, hence, includes faulty data as well as inconsistent data types.

In [8]:
# General imports
import os

import pandas as pd
import numpy as np
import spacy
import matplotlib
from wordcloud import WordCloud
from matplotlib import pyplot as plt

# Ensure matplotlib works correctly with Jupyter
%matplotlib inline

### 1. Data Loading (1 point)
Load the four datasets in the respective variables using pandas, and make sure to set the type of the `character_id` to integer.

In [ ]:
df_characters = ### Your code
df_locations = ### Your code
df_script = ### Your code
df_episodes = ### Your code

Drop all lines associated with the (non-numeric) character ID `142024`.

In [9]:
### Your code

Next, use panda's `.head()` function to inspect the top rows of each dataframe.

In [10]:
### Your code

### 2. Prominent Characters (4 points)

Let us now take a look at the most important characters of the show. To start, retrieve the most prominent characters by the number of spoken words/lines throughout the show. 

Extract the top 10 characters based on the number of spoken words (word count), as well as the 10 most prominent characters regarding their sum total number of lines.

You might notice that there is a column with `word_count` in the `df_scripts` data frame, but cleaning it up might be tricky. To find the sum of all values, first remove the `nan` entries and any other string characters in that column. You can use the provided `conv` function for this.

In [ ]:
def conv(row):
    try:
        return int(row)
    except:
        return np.nan
# First apply the conv() filtering on the word count
df_script["word_count"] = ### Your code ###
# And then drop all `nan` values in "word_count"
# Hint: Make sure the operation is done in-place.
### Your code ###

In [ ]:
# Now aggregate the top characters by words:
top_characters_by_words = ### your code

# And do the same by lines:
top_characters_by_lines = ### your code


Compare both results by drawing two bar plots: For the first plot, the x-axis should show the name of the top 10 characters with regard to the word count and the y-axis should display the number of spoken words.

In the second plot, you should do the same, but this time considering the number of lines (not words) for each character. Note that the characters shown in the two plots might differ.


In [11]:
# First plot: Top 10 characters by word count
# Hint: Try to use the integration of matplotlib and pandas to directly plot from a dataframe.

In [12]:
# Second plot: Top 10 characters by lines

Compare the results of the two plots: What do you notice? Do the results look correct to you?

### Your answer here

### 3. Common Words (3 points)

We continue by examining the dialogues in more detail, by looking at the top 20 most commonly appearing words (tokens).

To generate a meaningful output, use the following pre-processing pipeline, parts of which were explained in more detail during the tutorials and the lecture:
- Use spaCy (`en_core_web_sm`) to process all text.
- Instead of considering the raw text, use the lemmatized token provided by spaCy.
- Additionally, ignore any token that is classified as punctuation by spaCy.
- Also skip any token which is classified as a stop word by spaCy.

In [14]:
# Load the spacy model specified above. Make sure to disable the "ner" component for more speed.
nlp = spacy.###Your code here

word_list =[]

# Iterate over the spoken words (Hint: df_script) and append the lemmatized tokens as detailed above
### Your code here


word_counter = Counter(word_list)
counter.most_common(20)

SyntaxError: invalid syntax (4292105624.py, line 2)

Now plot a graph with the top 20 words on the x-axis and their respective frequencies on the y-axis, similar to the previous question.

In [16]:
# Provide a plot of the frequencies of the most common 20 words.

### 4. Named Entity Wordcloud (2 points)

Apply named entity recognition (NER) using SpaCy and generate a word cloud of the most commonly appearing 50
named entities that have the type `PERSON`. Visualize the resulting entities in a word cloud.
Take into account that you cannot simply use the 
output of the previous task, since NER was explicitly disabled.
 Since the named entity recognition takes quite some time, use only the first `10000` scripts.

In [ ]:
# Load the English spacy model again, but this time do not disable the NER module.
nlp = ### Your code

# Extract named entities of the type "PERSON" from each script.
for script in df_script.spoken_words.tolist()[:10000]:
    ### Your code

To visualize the word cloud, you may use the `wordcloud` package, which works in combination with `matplotlib`. Make sure to limit the number of visible words to 50, and the maximum font size to 60.

In [ ]:
# Create matplotlib skeleton for word cloud figure
fig = plt.figure(1, figsize=(12,12))
plt.axes("off")

cloud = ### Your code

plt.imshow(cloud, interpolation="bilinear")

### 5. Normalized Word Importance with TF-IDF (6 points)

The top words so far seem not too informative. We wish to find out the importance of particular words for specific characters, by looking at normalized frequency counts.
As an example, we expect the younger Simpson family members to exhibit a disproportionate frequency for the word "mom" or "dad", compared to other characters.

We looked at the normalization through Term Frequency - Inverse Document Frequency (TF-IDF) in the lecture, which we will use here to investigate lines by `Lisa Simpson` and `Bart Simpson`.

In this case, each script line is a document for a certain character who spoke the words.
Use `CountVectorizer` and `TfidfTransformers` from the `scikit-learn` library. In addition, filter out stop words and words with a frequency of less than 5 occurrences from the analysis. Then plot the TF-IDF values for the top 10 words for Lisa and Bart as bar plots.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

### Your code

In [ ]:
# Plot the top 10 occurrences, weighted by TF-IDF.

Do your results look correct?

### Your answer here

By using `CountVectorizer` over the concatenation of all character-specific lines, we can then also compute a (non-normalized) document similarity between the lines of Lisa and Bart. Use cosine similarity over the two document vectors to compare the similarity.

For the `CountVectorizer`, use the same arguments as before (minimum 5 occurrences and no stop words).

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

### Your code

### 6. NNMF Topics (5 points)

Uncover interesting topics by using non-negative matrix factorization (NNMF or NMF).

Use the `NMF` implementation from `sklearn` to detect the topics in the scripts, and use text cleaning steps similar to the first task. However, this time perform no lemmatization:
- Use spaCy (`en_core_web_sm`) to process all text.
- Additionally, ignore any token that is classified as punctuation by spaCy.
- Also skip any token which is classified as a stop word by spaCy.

Set the `max_features` of `CountVectorizer` to 10,000 and `random_state` of the `NMF` to 42, and `n_components=10`.

In [20]:
from sklearn.decomposition import NMF 

In [ ]:
# Clean the documents according to the specifications

In [ ]:
# Apply the NNMF method as specified above

In [22]:
def show_topics(components,num_top_words,vocab):
    #for each component or topic sorts the row values from large to small and 
    # returns the top words as the representation of the topic.
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in components])
    return [' '.join(t) for t in topic_words]

Use the `show_topics` method to show the top-20 words for the top 2 topics.

In [ ]:
### Your code

## Question 2-2: Boosting results in Elasticsearch (1 + 9 + 2 = 12 points)

Before you start, make sure to download the file `test_data.json`, available on [Moodle](https://moodle.uni-heidelberg.de/pluginfile.php/1006481/mod_label/intro/test-data.json). This question assumes you have a running installation of Elasticsearch 7.X on your machine. If not, make sure to follow the steps detailed in the first tutorial to set up an instance. To complete the following questions, you will have to adjust the given URLs with your own settings. Look out for mentions of `<login>`, `<password>` and `<port>` and replace them accordingly.

Bulk load the content of `test_data.json` into an index called `dishes`. You may use the following shell command:

```bash
curl -u <login>:<password> -H "Content-Type:application/x-ndjson" -XPOST http://localhost:<port>/dishes/_bulk --data-binary "@test_data.json"
```
1. Provide a command to extract the dynamic mapping that is created in this scenario. Make sure that the output mapping is included in your solution (potentially pasting it into another cell or execute through the ).

### Insert your answer here
Hint: Consider using the option `?pretty` for legible outputs

GET /dishes/_mapping?pretty=true

{
  "dishes" : {
    "mappings" : {
      "properties" : {
        "created" : {
          "type" : "date",
          "format" : "yyyy/MM/dd HH:mm:ss||yyyy/MM/dd||epoch_millis"
        },
        "description" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        },
        "ingredients" : {
          "properties" : {
            "name" : {
              "type" : "text",
              "fields" : {
                "keyword" : {
                  "type" : "keyword",
                  "ignore_above" : 256
                }
              }
            },
            "quantity" : {
              "type" : "text",
              "fields" : {
                "keyword" : {
                  "type" : "keyword",
                  "ignore_above" : 256
                }
              }
            }
          }
        },
        "preparation_time_minutes" : {
          "type" : "long"
        },
        "ratings" : {
          "type" : "float"
        },
        "servings" : {
          "properties" : {
            "max" : {
              "type" : "long"
            },
            "min" : {
              "type" : "long"
            }
          }
        },
        "steps" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        },
        "title" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        }
      }
    }
  }
}


2. Write queries to the `dishes` index given the specified criteria. The matching documents should be included in your solution, including their respective scores.

a) Return all dishes that have a preparation time of less than 15 minutes

### Insert your answer here

GET /dishes/_search
{
  "_source": [
    "title",
    "preparation_time_minutes"
  ],
  "query":{
    "range": {
      "preparation_time_minutes": {
        "lt": 15
      }
    }
  }
}

{
  "took" : 3,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 3,
      "relation" : "eq"
    },
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "dishes",
        "_type" : "_doc",
        "_id" : "1",
        "_score" : 1.0,
        "_ignored" : [
          "description.keyword",
          "steps.keyword"
        ],
        "_source" : {
          "preparation_time_minutes" : 12,
          "title" : "Fast and Easy Pasta With Blistered Cherry Tomato Sauce"
        }
      },
      {
        "_index" : "dishes",
        "_type" : "_doc",
        "_id" : "5",
        "_score" : 1.0,
        "_ignored" : [
          "description.keyword",
          "steps.keyword"
        ],
        "_source" : {
          "preparation_time_minutes" : 8,
          "title" : "Stovetop Macaroni and Cheese"
        }
      },
      {
        "_index" : "dishes",
        "_type" : "_doc",
        "_id" : "6",
        "_score" : 1.0,
        "_ignored" : [
          "steps.keyword"
        ],
        "_source" : {
          "preparation_time_minutes" : 10,
          "title" : "Spaghetti Aglio e Olio Recipe"
        }
      }
    ]
  }
}


b) All dishes that contain `spaghetti` in the title.

### Insert your answer here

GET /dishes/_search
{
  "_source": [
    "title"
  ],
  "query": {
    "match": {
      "title": {
        "query": "spaghetti"
      }
    }
  }
}

{
  "took" : 3,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 4,
      "relation" : "eq"
    },
    "max_score" : 1.9130536,
    "hits" : [
      {
        "_index" : "dishes",
        "_type" : "_doc",
        "_id" : "11",
        "_score" : 1.9130536,
        "_ignored" : [
          "description.keyword",
          "steps.keyword"
        ],
        "_source" : {
          "title" : "Spaghetti Puttanesca (Pasta or Spaghetti With Capers, Olives, and Anchovies)"
        }
      },
      {
        "_index" : "dishes",
        "_type" : "_doc",
        "_id" : "6",
        "_score" : 1.7677585,
        "_ignored" : [
          "steps.keyword"
        ],
        "_source" : {
          "title" : "Spaghetti Aglio e Olio Recipe"
        }
      },
      {
        "_index" : "dishes",
        "_type" : "_doc",
        "_id" : "4",
        "_score" : 1.4669425,
        "_ignored" : [
          "description.keyword",
          "steps.keyword"
        ],
        "_source" : {
          "title" : "Shrimp Fra Diavolo (Spaghetti With Spicy Tomato Sauce)"
        }
      },
      {
        "_index" : "dishes",
        "_type" : "_doc",
        "_id" : "7",
        "_score" : 1.3174803,
        "_ignored" : [
          "description.keyword",
          "steps.keyword"
        ],
        "_source" : {
          "title" : "Cacio e Pepe (Spaghetti With Black Pepper and Pecorino Romano)"
        }
      }
    ]
  }
}


c) All dishes that received at least one 5-star rating and were created after 2016.

### Insert your answer here

GET /dishes/_search 
{
  "_source": [
    "title",
    "ratings",
    "created"
  ],
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "ratings": "5.0"
          } 
        },
        {
          "range": {
            "created": {
              "gte": "2017/01/01", 
              "lte": "now"
            }
          }
        }
      ]
    }
  }
}

{
  "took" : 2,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 2,
      "relation" : "eq"
    },
    "max_score" : 2.0,
    "hits" : [
      {
        "_index" : "dishes",
        "_type" : "_doc",
        "_id" : "1",
        "_score" : 2.0,
        "_ignored" : [
          "description.keyword",
          "steps.keyword"
        ],
        "_source" : {
          "created" : "2017/03/29",
          "ratings" : [
            4.5,
            5.0,
            3.0,
            4.5
          ],
          "title" : "Fast and Easy Pasta With Blistered Cherry Tomato Sauce"
        }
      },
      {
        "_index" : "dishes",
        "_type" : "_doc",
        "_id" : "10",
        "_score" : 2.0,
        "_ignored" : [
          "steps.keyword"
        ],
        "_source" : {
          "created" : "2017/04/27",
          "ratings" : [
            1.5,
            2.0,
            4.0,
            3.5,
            3.0,
            5.0,
            1.5
          ],
          "title" : "Penne With Hot-As-You-Dare Arrabbiata Sauce"
        }
      }
    ]
  }
}


d) Dishes that serve between 6 to 8 people.

### Insert your answer here

GET /dishes/_search 
{
  "_source": [
    "title", 
    "servings.max",
    "servings.min"
  ],
  "query": {
    "bool": {
      "must": [
        {
          "range": {
            "servings.min": {
              "gte": 6
            }
          }
          },
          {
          "range": {
            "servings.max": {
              "lte": 8
            }
          }
          }
      ]
    }
  }
}

{
  "took" : 2,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 1,
      "relation" : "eq"
    },
    "max_score" : 2.0,
    "hits" : [
      {
        "_index" : "dishes",
        "_type" : "_doc",
        "_id" : "20",
        "_score" : 2.0,
        "_ignored" : [
          "description.keyword",
          "steps.keyword"
        ],
        "_source" : {
          "servings" : {
            "min" : 6,
            "max" : 8
          },
          "title" : "Easy Skillet Baked Ziti With Sausage and Ricotta"
        }
      }
    ]
  }
}


e) Dishes that mention both of the phrases "pasta" and "parmigiano" in the instructions.

### Insert your answer here

GET /dishes/_search 
{
  "_source": [
    "title", 
    "steps"
  ],
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "steps": "pasta"
          }
        },
        {
          "match": {
            "steps": "parmigiano"
          }
        }
      ]
    }
  }
}

{
  "took" : 5,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 3,
      "relation" : "eq"
    },
    "max_score" : 2.3426425,
    "hits" : [
      {
        "_index" : "dishes",
        "_type" : "_doc",
        "_id" : "13",
        "_score" : 2.3426425,
        "_ignored" : [
          "steps.keyword"
        ],
        "_source" : {
          "title" : "Pesto Pasta With Potatoes and Green Beans",
          "steps" : [
            "In a large pot of salted boiling water, boil pasta, potato, and green beans until pasta is al dente and potato and green beans are very tender. Drain, reserving 1 cup cooking water, and transfer pasta, potato, and green beans to a large mixing or serving bowl.",
            "Add pesto sauce to pasta along with 1/4 cup pasta cooking water. Toss well to emulsify pesto and pasta water into a creamy sauce. Add more pasta water, 1 tablespoon at a time, as needed, if pasta is too dry. Drizzle in fresh olive oil, if desired. Serve with Parmigiano Reggiano on the side."
          ]
        }
      },
      {
        "_index" : "dishes",
        "_type" : "_doc",
        "_id" : "20",
        "_score" : 2.1880798,
        "_ignored" : [
          "description.keyword",
          "steps.keyword"
        ],
        "_source" : {
          "title" : "Easy Skillet Baked Ziti With Sausage and Ricotta",
          "steps" : [
            "Place pasta in a large bowl and cover with hot water. Season generously with salt. Let rest, stirring twice during the first 10 minutes, while you prepare the other ingredients.",
            "Use a hand blender or countertop blender to process tomatoes until mostly smooth, but still a little chunky. Set aside 3/4 cup of tomatoes. Combine remaining tomatoes, heavy cream, and chicken stock in a medium bowl. Season to taste with salt and set aside.",
            "Heat oil and butter in a large straight-sided sauté pan or Dutch oven over medium-high heat, swirling, until butter is mostly melted. Add sausage and cook, mashing with a potato masher or a whisk, until sausage is no longer pink, about 5 minutes. Add onion and garlic, reduce heat to medium, and cook, stirring frequently, until softened but not browned, about 5 minutes. Add oregano, red pepper flakes, and half of parsley and cook, stirring, until fragrant, about 1 minute.",
            "Add tomato and cream mixture to pan with sausage. Drain noodles in a large colander set in the sink, then add to pan and stir to combine. Stir in half of ricotta, then rapidly stir in half of mozzarella cheese. (Do not over-stir, or the mixture will stretch and stick to your spoon.) Spoon reserved 3/4 cup tomatoes over top of pasta. Dollop with remaining ricotta and scatter remaining mozzarella over top. Sprinkle with half of Parmigiano-Reggiano. Cover and cook over the lowest possible heat for 3 minutes. Remove from heat and let rest, covered, for 5 minutes.",
            "Uncover, sprinkle with remaining Parmigiano-Reggiano and parsley, and serve immediately."
          ]
        }
      },
      {
        "_index" : "dishes",
        "_type" : "_doc",
        "_id" : "2",
        "_score" : 1.8295534,
        "_ignored" : [
          "description.keyword",
          "steps.keyword"
        ],
        "_source" : {
          "title" : "Pasta With Butternut Squash and Sage Brown Butter",
          "steps" : [
            "Heat olive oil in a large stainless steel or cast-iron skillet over high heat until very lightly smoking. Immediately add squash, season with salt and pepper, and cook, stirring and tossing occasionally, until well-browned and squash is tender, about 5 minutes. Add butter and shallots and continue cooking, stirring frequently, until butter is lightly browned and smells nutty, about 1 minute longer. Add sage and stir to combine (sage should crackle and let off a great aroma). Remove from heat and stir in lemon juice. Set aside.",
            "In a medium saucepan, combine pasta with enough room temperature or hot water to cover by about 2 inches. Season with salt. Set over high heat and bring to a boil while stirring frequently. Cook, stirring frequently, until pasta is just shy of al dente, about 2 minutes less than the package directions. Drain pasta, reserving a couple cups of the starchy cooking liquid.",
            "Add pasta to skillet with squash along with a splash of pasta water. Bring to a simmer over high heat and cook until the pasta is perfectly al dente, stirring and tossing constantly and adding a splash of water as needed to keep the sauce loose and shiny. Off heat, stir in Parmigiano-Reggiano. Adjust seasoning with salt and pepper and texture with more pasta water as needed. Serve immediately, topped with more cheese at the table."
          ]
        }
      }
    ]
  }
}

f) All recipes that require more than 10 ingredients

### Insert your answer here

GET dishes/_search
{
  "_source": [
    "title", 
    "ingredients.name",
    "ingredients.quantity"
  ],
  "query": {
    "bool": {
      "must": {
        "script": {
          "script": {
            "source": "doc['ingredients.name.keyword'].length > 10",
            "lang": "painless"
          }
        }
      }
    }
  }
}

{'took': 13, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 13, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'dishes', '_type': '_doc', '_id': '3', '_score': 1.0, '_ignored': ['description.keyword', 'steps.keyword'], '_source': {'ingredients': [{'quantity': '1 tablespoon', 'name': 'Extra-virgin olive oil'}, {'quantity': '115g', 'name': 'Thinly sliced prosciutto'}, {'quantity': '1/4 cup', 'name': 'Thinly sliced green garlic'}, {'quantity': '2', 'name': 'Cloves garlic'}, {'quantity': '450g', 'name': 'Asparagus'}, {'quantity': '1 cup', 'name': 'Heavy cream'}, {'quantity': '55g', 'name': 'Grated Parmigiano-Reggiano cheese'}, {'name': 'Kosher salt'}, {'name': 'Freshly ground black pepper'}, {'quantity': '1 tablespoon', 'name': 'Lemon juice'}, {'quantity': '1 teaspoon', 'name': 'grated zest'}, {'quantity': '2 tablespoons', 'name': 'Minced fresh chives'}, {'name': 'Ricotta'}], 'title': 'Ricotta Gnocchi With Asparagus and Prosciutto'}}, {'_index': 'dishes', '_type': '_doc', '_id': '4', '_score': 1.0, '_ignored': ['description.keyword', 'steps.keyword'], '_source': {'ingredients': [{'quantity': '1/2 teaspoon', 'name': 'Kosher salt'}, {'quantity': '340g', 'name': 'Large shrimp'}, {'name': 'Large pinch baking soda'}, {'quantity': '90ml', 'name': 'Extra-virgin olive oil'}, {'quantity': '4', 'name': 'Cloves garlic'}, {'quantity': '1,5 teaspoons', 'name': 'Dried oregano'}, {'quantity': '1-2 teaspoons', 'name': 'Red chili flakes '}, {'quantity': '30ml', 'name': 'Brandy'}, {'quantity': '800g', 'name': 'Whole peeled tomatoes'}, {'quantity': '120ml', 'name': 'Clam juice'}, {'quantity': '450g', 'name': 'Spaghetti'}, {'quantity': '1/4 cup', 'name': 'Minced flat-leaf parsley leaves'}], 'title': 'Shrimp Fra Diavolo (Spaghetti With Spicy Tomato Sauce)'}}, {'_index': 'dishes', '_type': '_doc', '_id': '8', '_score': 1.0, '_ignored': ['description.keyword', 'steps.keyword'], '_source': {'ingredients': [{'quantity': '200g', 'name': 'Silken tofu'}, {'quantity': '120ml', 'name': 'Sauerkraut brine'}, {'quantity': '15g', 'name': 'Nutritional yeast'}, {'quantity': '15ml', 'name': 'White miso'}, {'name': 'Cayenne pepper'}, {'quantity': '1/4 teaspoon', 'name': 'Smoked paprika'}, {'quantity': '2 teaspoons', 'name': 'Freshly ground black pepper'}, {'quantity': '1 teaspoon', 'name': 'White wine vinegar'}, {'name': 'Kosher salt'}, {'quantity': '120ml', 'name': 'Extra-virgin olive oil'}, {'quantity': '115g', 'name': 'King oyster mushrooms'}, {'quantity': '450g', 'name': 'Dry spaghetti or penne'}], 'title': 'Vegan Carbonara Pasta'}}, {'_index': 'dishes', '_type': '_doc', '_id': '11', '_score': 1.0, '_ignored': ['description.keyword', 'steps.keyword'], '_source': {'ingredients': [{'quantity': '225g', 'name': 'Dried spaghetti'}, {'name': 'Kosher salt'}, {'quantity': '6 tablespoons', 'name': 'Extra-virgin olive oil'}, {'quantity': '4', 'name': 'Cloves garlic'}, {'quantity': '4-6', 'name': 'Anchovy fillets'}, {'name': 'Large pinch red pepper flakes'}, {'quantity': '1/4 cup', 'name': 'Capers'}, {'quantity': '1/4 cup', 'name': 'Pitted black olives'}, {'quantity': '225g', 'name': 'Whole peeled tomatoes'}, {'quantity': 'Small handful', 'name': 'Minced fresh parsley leaves'}, {'quantity': '30g', 'name': 'Finely grated Pecorino Romano or Parmesan cheese'}, {'name': 'Freshly ground black pepper'}, {'quantity': '140g', 'name': 'Can oil-packed tuna'}], 'title': 'Spaghetti Puttanesca (Pasta or Spaghetti With Capers, Olives, and Anchovies)'}}, {'_index': 'dishes', '_type': '_doc', '_id': '12', '_score': 1.0, '_ignored': ['description.keyword', 'steps.keyword'], '_source': {'ingredients': [{'name': 'Kosher salt'}, {'quantity': '1', 'name': 'Large russet potato'}, {'quantity': '2', 'name': 'Medium carrots'}, {'quantity': '85g', 'name': 'String beans'}, {'quantity': '1/2', 'name': 'Small fennel bulb'}, {'quantity': '1', 'name': 'Small red onion'}, {'quantity': '450g', 'name': 'Dried penne'}, {'quantity': '2-4', 'name': 'Cloves garlic'}, {'quantity': '3/4 cup', 'name': 'Extra-virgin olive oil'}, {'quantity': '1/4 cup', 'name': 'Minced parsley'}, {'name': 'Freshly ground black pepper'}, {'name': 'Grated Parmigiano-Reggiano'}], 'title': 'Penne With Melted-Vegetable Sauce'}}, {'_index': 'dishes', '_type': '_doc', '_id': '14', '_score': 1.0, '_ignored': ['description.keyword', 'steps.keyword'], '_source': {'ingredients': [{'quantity': '140g', 'name': 'Grated Parmigiano Reggiano cheese'}, {'quantity': '30ml', 'name': 'Heavy cream'}, {'quantity': '1', 'name': 'Egg'}, {'quantity': '1 teaspoon', 'name': 'Cornstarch'}, {'quantity': '2 tablespoons', 'name': 'Extra-virgin olive oil'}, {'quantity': '1/2 teaspoons', 'name': 'Grated lemon zest'}, {'name': 'Kosher salt'}, {'name': 'Freshly ground black pepper'}, {'quantity': '450g', 'name': 'Fresh fettuccine'}, {'quantity': '1 teaspoon', 'name': 'Minced garlic'}, {'quantity': '30g', 'name': 'Unsalted butter'}, {'name': 'Minced fresh parsley'}], 'title': 'Lighter Fettuccine Alfredo Recipe'}}, {'_index': 'dishes', '_type': '_doc', '_id': '15', '_score': 1.0, '_ignored': ['description.keyword', 'steps.keyword'], '_source': {'ingredients': [{'quantity': '170g', 'name': 'Dried wide egg noodles'}, {'name': 'Kosher salt'}, {'quantity': '170g', 'name': 'Crème fraîche'}, {'quantity': '1', 'name': 'Egg'}, {'quantity': '2 teaspoons', 'name': 'Cornstarch'}, {'name': 'Freshly ground black pepper'}, {'quantity': '2 tablespoons', 'name': 'Lemon juice'}, {'quantity': '1 can', 'name': 'Tuna'}, {'quantity': '1 cup', 'name': 'Frozen peas'}, {'quantity': '1/4 cup', 'name': 'Picked fresh parsley leaves'}, {'name': 'Extra-virgin olive oil'}, {'name': 'Crushed potato chips'}], 'title': 'Lighter Tuna Noodle Casserole'}}, {'_index': 'dishes', '_type': '_doc', '_id': '16', '_score': 1.0, '_ignored': ['description.keyword', 'steps.keyword'], '_source': {'ingredients': [{'quantity': '2 tablespoons', 'name': 'Extra-virgin olive oil'}, {'quantity': '2 cups', 'name': 'Oyster mushrooms'}, {'name': 'Kosher salt'}, {'name': 'Freshly ground black pepper'}, {'quantity': '1', 'name': 'Shallot'}, {'quantity': '2', 'name': 'Cloves garlic'}, {'name': 'Dried red chili flakes'}, {'quantity': '1/4 cup', 'name': 'Dry white wine'}, {'quantity': '450g', 'name': 'Dried orecchiette'}, {'quantity': '950ml', 'name': 'Homemade chicken stock'}, {'quantity': '2 cups', 'name': 'Spinach'}, {'quantity': '450g', 'name': 'Medium shrimp'}, {'quantity': '2 tablespoons', 'name': 'Lemon juice'}], 'title': 'One-Skillet Orecchiette With Shrimp, Spinach, and Mushrooms'}}, {'_index': 'dishes', '_type': '_doc', '_id': '17', '_score': 1.0, '_ignored': ['description.keyword', 'steps.keyword'], '_source': {'ingredients': [{'quantity': '1/2 tablespoon', 'name': 'Extra-virgin olive oil'}, {'quantity': '115g', 'name': 'Panchetta'}, {'quantity': '2 cups', 'name': 'Shiitake mushrooms'}, {'quantity': '1', 'name': 'Shallot'}, {'quantity': '1/2', 'name': 'Serrano chile pepper'}, {'quantity': '5 cups', 'name': 'Homemade chicken or vegetable stock'}, {'quantity': '450g', 'name': 'Dried fusilli'}, {'name': 'Kosher salt'}, {'name': 'Freshly ground black pepper'}, {'quantity': '2 tablespoons', 'name': 'Fresh lemon juice'}, {'quantity': 'A bunch', 'name': 'Greens (e.g. kale)'}, {'name': 'Parmesan'}], 'title': 'Skillet Pasta With Mushrooms, Pancetta, and Wilted Greens'}}, {'_index': 'dishes', '_type': '_doc', '_id': '18', '_score': 1.0, '_ignored': ['description.keyword', 'steps.keyword'], '_source': {'ingredients': [{'quantity': '2 tablespoons', 'name': 'Olive oil'}, {'quantity': 'A bunch', 'name': 'Swiss chard'}, {'quantity': '1', 'name': 'Large shallot'}, {'name': 'Kosher salt'}, {'name': 'Freshly ground black pepper'}, {'name': 'Dried red chile flakes'}, {'quantity': '4,5 cups', 'name': 'Homemade vegetable stock'}, {'quantity': '450g', 'name': 'Whole wheat fusilli pasta'}, {'quantity': '2 tablespoons', 'name': 'Fresh lemon juice'}, {'quantity': '1 tablespoon', 'name': 'Sumac'}, {'name': 'Parmigiano-Reggiano'}], 'title': 'Vegetarian Citrus Pasta With Swiss Chard'}}, {'_index': 'dishes', '_type': '_doc', '_id': '19', '_score': 1.0, '_ignored': ['steps.keyword'], '_source': {'ingredients': [{'quantity': '3 tablespoons', 'name': 'Olive oil'}, {'quantity': '115g', 'name': 'Brussels sprouts'}, {'name': 'Kosher salt'}, {'name': 'Freshly ground black pepper'}, {'quantity': '225g', 'name': 'Mixed mushrooms'}, {'quantity': '1', 'name': 'Shallot'}, {'quantity': '2', 'name': 'Cloves garlic'}, {'quantity': '1 teaspoon', 'name': 'Fresh thyme leaves'}, {'quantity': '4 tablespoons', 'name': 'Butter'}, {'quantity': '2 teaspoons', 'name': 'Freshly squeezed lemon juice'}, {'quantity': '1/2 cup', 'name': 'Homemade vegetable or chicken stock'}, {'quantity': '450g', 'name': 'Dried orecchiette'}, {'quantity': '55g', 'name': 'Freshly grated Parmesan cheese'}], 'title': 'Pasta With Mushrooms, Brussels Sprouts, and Parmesan'}}, {'_index': 'dishes', '_type': '_doc', '_id': '20', '_score': 1.0, '_ignored': ['description.keyword', 'steps.keyword'], '_source': {'ingredients': [{'quantity': '450g', 'name': 'Dry ziti or penne'}, {'name': 'Kosher salt'}, {'quantity': '1', 'name': 'Can whole peeled tomatoes'}, {'quantity': '1 cup', 'name': 'Heavy cream'}, {'quantity': '1 cup', 'name': 'Homemade chicken stock'}, {'quantity': '2 tablespoons', 'name': 'Extra-virgin olive oil'}, {'quantity': '2 tablespoons', 'name': 'Unsalted butter'}, {'quantity': '450g', 'name': 'Italian sausage'}, {'quantity': '1', 'name': 'Large onion'}, {'quantity': '4', 'name': 'Cloves garlic'}, {'quantity': '1 tablespoon', 'name': 'Dried oregano'}, {'quantity': '1 teaspoon', 'name': 'Crushed red pepper flakes'}, {'quantity': '1/4 cup', 'name': 'Finely minced fresh parsley leaves'}, {'quantity': '350g', 'name': 'Ricotta cheese'}, {'quantity': '450g', 'name': 'Low-moisture whole-milk mozzarella cheese'}, {'quantity': '85g', 'name': 'Roughly grated Parmigiano-Reggiano'}], 'title': 'Easy Skillet Baked Ziti With Sausage and Ricotta'}}, {'_index': 'dishes', '_type': '_doc', '_id': '21', '_score': 1.0, '_ignored': ['description.keyword', 'steps.keyword'], '_source': {'ingredients': [{'quantity': '1 cup', 'name': 'Panko-style bread crumbs'}, {'quantity': '175g', 'name': 'Grated Parmesan cheese'}, {'quantity': '1/4 cup', 'name': 'Chopped fresh parsley leaves'}, {'quantity': '2 tablespoons', 'name': 'Finely minced fresh chives'}, {'quantity': '2', 'name': 'Shallots'}, {'quantity': '2', 'name': 'Cloves garlic'}, {'quantity': '2 tablespoons', 'name': 'Extra-virgin olive oil'}, {'name': 'Kosher salt'}, {'name': 'Freshly ground black pepper'}, {'quantity': '2 tablespoons', 'name': 'Unsalted butter'}, {'quantity': '225g', 'name': 'Italian sausage'}, {'quantity': '450g', 'name': 'Mixed mushrooms'}, {'quantity': '1 tablespoon', 'name': 'Soy sauce'}, {'quantity': '1 tablespoon', 'name': 'Fresh lemon juice'}, {'quantity': '2,5 tablespoons', 'name': 'Flour'}, {'quantity': '2 cups', 'name': 'Homemade chicken stock'}, {'quantity': '1 cup', 'name': 'Heavy cream'}, {'quantity': '350g', 'name': 'Fresh ridged pasta'}], 'title': 'Crispy Baked Pasta With Mushrooms, Sausage, and Parmesan Cream Sauce'}}]}}


g) Extend your query from b), such that it searches for `spaghetti` in both the title *and* description field. Occurrences in the title field should be boosted by a factor of 3.

### Insert your answer here

GET dishes/_search
{
  "_source": [
    "title", 
    "description"
  ],
  "query": {
    "bool": {
      "should": [
        { "match": { "description": "spaghetti"} },
        { "match": { "title": {
                      "query": "spaghetti",
                      "boost": 3}
                    }}
      ]
    }
  }
}

{
  "took" : 7,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : {
      "value" : 4,
      "relation" : "eq"
    },
    "max_score" : 7.7521906,
    "hits" : [
      {
        "_index" : "dishes",
        "_type" : "_doc",
        "_id" : "7",
        "_score" : 7.7521906,
        "_ignored" : [
          "description.keyword",
          "steps.keyword"
        ],
        "_source" : {
          "description" : "If you were to watch a practiced hand make cacio e pepe, you might think the instructions were as simple as this: Cook spaghetti and drain. Toss with olive oil, butter, black pepper, and grated Pecorino Romano cheese. Serve. But we all know that the simplest recipes can often be the most confounding, and so it is with cacio e pepe. Follow those instructions and, if you're lucky, you'll get what you're after: a creamy, emulsified sauce that coats each strand of spaghetti with flavor. More likely, you're gonna get what I (and, from the stories I've heard, many others as well) got on the first few tries—spaghetti in a thin, greasy sauce, or spaghetti with clumps of cheese that refuse to melt. Or, worse, both at the same time. Here's how to make it perfectly every time.",
          "title" : "Cacio e Pepe (Spaghetti With Black Pepper and Pecorino Romano)"
        }
      },
      {
        "_index" : "dishes",
        "_type" : "_doc",
        "_id" : "11",
        "_score" : 5.7391615,
        "_ignored" : [
          "description.keyword",
          "steps.keyword"
        ],
        "_source" : {
          "description" : "\"Puttanesca\" literally translates to \"in the style of prostitutes,\" supposedly because the pungent aromas of garlic, anchovies, capers, and olives tossed with pasta were how Neapolitan prostitutes would lead customers to their doors. This is one of those stories that seem, in the words of Douglas Adams, apocryphal or at least wildly inaccurate. That said, it's a fitting title—puttanesca packs an aromatic punch and then some.",
          "title" : "Spaghetti Puttanesca (Pasta or Spaghetti With Capers, Olives, and Anchovies)"
        }
      },
      {
        "_index" : "dishes",
        "_type" : "_doc",
        "_id" : "6",
        "_score" : 5.303276,
        "_ignored" : [
          "steps.keyword"
        ],
        "_source" : {
          "description" : "One of the most basic pasta sauces, aglio e olio uses just garlic and olive oil (and maybe a pinch of red pepper flakes for heat). It sounds too simple to be good, but it's among the best.",
          "title" : "Spaghetti Aglio e Olio Recipe"
        }
      },
      {
        "_index" : "dishes",
        "_type" : "_doc",
        "_id" : "4",
        "_score" : 4.400828,
        "_ignored" : [
          "description.keyword",
          "steps.keyword"
        ],
        "_source" : {
          "description" : "Lobster fra diavolo is a classic Italian-American pasta dish, but the lobster version is a lot easier for restaurants than home cooks. Shrimp make an excellent stand-in, as long as you know how to infuse the spicy tomato sauce with some real shellfish flavor.",
          "title" : "Shrimp Fra Diavolo (Spaghetti With Spicy Tomato Sauce)"
        }
      }
    ]
  }
}


3. The default analyzer might not be perfect for working with recipe data. Provide the definition of a custom analyzer which adheres to the following specifications:
   
   - It performs *no* stop word filtering.
   - It applies stemming to all tokens.
   - It does *not* lowercase tokens, but keeps the original case of letters.

### Insert your answer here

PUT /dishes/_settings
{
    "analysis": {
      "analyzer": {
        "default": {
          "tokenizer": "standard",
          "filter": ["stemming_all_tokens_filter"]
        }
      },
      "filter": {
        "stemming_all_tokens_filter":{
          "type": "stemmer"
        }
      }
    }
}

## Question 2-3: Document Representations (2 + 2 + 3 = 7 points)

Please fill out the answers directly in the provided fields.

#### a) Why is it preferable to use cosine distance over Euclidean distance when computing document similarity?

Answer:

#### b) Identify two shortcomings of Bag-of-Word-based document representations

Answer:

#### c) Find the singular value decomposition of the following matrix:

$A = \left[ \begin{matrix}1&-1\\ -2&2\\ 2&-2\end{matrix} \right]$.


Answer: